# 텍스트 마이닝 기법 기반 데헬스 앱 리뷰
#### 👨‍💻 Author: Young Kim

# 패키지 설치

In [ ]:
# Mecab 설치
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# mecab-python의 버전 오류로 인해 아래 패키지를 설치하면 코랩에서 Mecab을 사용가능
!pip install mecab-python3

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,064 kB]
Get:14 

In [ ]:
# Install the pyLADvis package
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 20.8 MB/s eta 0:00:00


# 패키지 import

In [ ]:
import numpy as np
import pandas as pd
import warnings # 경고 메시지 무시
warnings.filterwarnings(action='ignore')
# 한국어 형태소 분석기 중 성능이 가장 우수한 Mecab 사용
from konlpy.tag import Mecab
mecab = Mecab()
from tqdm import tqdm # 작업 프로세스 시각화
import re # 문자열 처리를 위한 정규표현식 패키지
from gensim import corpora # 단어 빈도수 계산 패키지
import gensim # LDA 모델 활용 목적
import pyLDAvis.gensim_models # LDA 시각화용 패키지
from collections import Counter # 단어 등장 횟수 카운트

# 데이터셋 load

In [ ]:
dataset_raw = pd.read_csv('/content/thehealth_reviews_kr.csv')
dataset_raw.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,리뷰ID,리뷰,평점,평점등록일자,답변,답변등록일자
0,주야만세야,좋아요. 너무 좋은데 수면측정을 해도 안됩니다. 저만안되는 걸까요? 밤새도록 마이크...,5,5/13/2024 7:21,NaN,NaN
1,캉네,아이폰 사용자입니다. 식단에서 상세수정들어가면 화면 바깥으로 튕기네요. 그것빼고는 ...,5,12/10/2023 7:21,NaN,NaN
2,Jang Phil Soo,아이폰 어플 업데이트 좀 해주세요. 구글 앱은 잘 작동하는 것 같은데요. 아이폰 앱...,2,11/14/2023 7:21,NaN,NaN
3,장미향 쩌러,"모션운동 해보니 만족스럽긴한데 공간의 활용성, 핸드폰의 활용도 등 사용자의 여건에 ...",5,12/23/2023 0:00,NaN,NaN
4,최나니,다 좋은데 권한요청 관련 앱 개선 요청 드립니다. 회사 보안 때문에 회사에 들어오면...,5,5/13/2024 7:21,NaN,NaN


# 데이터 탐색
- 데이터셋에서 전반적으로 결측치 존재여부, 데이터 타입, 데이터 개수를 확인합니다.

In [ ]:
dataset_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   리뷰ID    461 non-null    object
 1   리뷰      461 non-null    object
 2   평점      461 non-null    int64 
 3   평점등록일자  461 non-null    object
 4   답변      261 non-null    object
 5   답변등록일자  261 non-null    object
dtypes: int64(1), object(5)
memory usage: 21.7+ KB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 데이터 전처리

## 결측치 확인

In [ ]:
dataset_raw.isnull().sum()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


리뷰ID        0
리뷰          0
평점          0
평점등록일자      0
답변        200
답변등록일자    200
dtype: int64

리뷰에 대한 답변이 없는 경우는 허용 가능하므로 결측지 제거를 실시하지 않음

In [ ]:
dataset = dataset_raw

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 전처리용 딕셔너리 Load

### 불용어 사전 리스트
- 사용자의 서비스 경험과는 무관하지만 빈출되는 단어인 불용어는 미리 제거하는 작업이 필요합니다.

In [ ]:
stopword_list = pd.read_excel('/content/stopword_list.xlsx')
stopword_list.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,stopword
0,가까스로
1,가량
2,가령
3,가민
4,가민커넥트


### 데이터 치환 리스트
- 같은 의미로 사용된 여러 데이터를 특정 단어로 통일화하기 위한 데이터셋입니다.
- 본 프로젝트에서는 특정 단어의 빈출여부가 핵심이기 때문에 같은 의미의 단어를 통일화하는 게 중요합니다.

In [ ]:
replace_list = pd.read_excel('/content/replace_list.xlsx')
replace_list.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,before_replacement,after_replacement
0,S헬스,삼성헬스
1,LG폰,스마트폰
2,LG V10,스마트폰
3,G7,스마트폰
4,GX,그룹운동


### 한 글자 키워드 리스트
- 일반적으로 토큰이 1글자인 경우는 대부분 불용어에 해당하기 때문에 제거합니다.
- 단, 프로젝트 성격에 따라 1글자의 단어가 핵심 키워드가 될 수 있습니다.
- 아래의 리스트는 이처럼 키워드로 생각되는 1글자 단어를 리스트업 하였습니다.

In [ ]:
one_char_keyword = pd.read_excel('/content/one_char_list.xlsx')
one_char_keyword.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,one_char_keyword
0,컵
1,방
2,물
3,돈
4,꿈


## 데이터 치환
- 같은 의미의 단어를 하나의 단어로 통일하는 작업입니다.
- LDA 토픽 모델링은 빈출 어휘를 중심으로 결과를 제공하기 때문에 단어를 통일할 필요가 있습니다.

In [ ]:
def replace_word(review):
    for i in range(len(replace_list['before_replacement'])):
        try:
            # 치환할 단어가 있는 경우에만 데이터 치환 수행
            if replace_list['before_replacement'][i] in review:
                review = review.replace(replace_list['before_replacement'][i], replace_list['after_replacement'][i])
        except Exception as e:
            print(f"Error 발생 / 에러명: {e}")
    return review

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
dataset['review_prep'] = ''
review_replaced_list = []
for review in tqdm(dataset['리뷰']):
    review_replaced = replace_word(str(review)) # 문자열 데이터 변환
    review_replaced_list.append(review_replaced)
dataset['review_prep'] = review_replaced_list
dataset.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 461/461 [00:01<00:00, 381.23it/s]


,리뷰ID,리뷰,평점,평점등록일자,답변,답변등록일자,review_prep
0,주야만세야,좋아요. 너무 좋은데 수면측정을 해도 안됩니다. 저만안되는 걸까요? 밤새도록 마이크...,5,5/13/2024 7:21,NaN,NaN,좋아요. 너무 좋습니다만 수면측정을 해도 안됩니다. 저만안되는 걸까요? 밤새도록 마...
1,캉네,아이폰 사용자입니다. 식단에서 상세수정들어가면 화면 바깥으로 튕기네요. 그것빼고는 ...,5,12/10/2023 7:21,NaN,NaN,아이폰 사용자입니다. 식단에서 상세수정들어가면 화면 바깥으로 튕기네요. 그것빼고는 ...
2,Jang Phil Soo,아이폰 어플 업데이트 좀 해주세요. 구글 앱은 잘 작동하는 것 같은데요. 아이폰 앱...,2,11/14/2023 7:21,NaN,NaN,아이폰 어플 업데이트 좀 해주세요. 구글 애플리케이션은 잘 작동하는 것 같은데요. ...
3,장미향 쩌러,"모션운동 해보니 만족스럽긴한데 공간의 활용성, 핸드폰의 활용도 등 사용자의 여건에 ...",5,12/23/2023 0:00,NaN,NaN,"모션운동 해보니 만족스럽긴한데 공간의 활용성, 스마트폰의 활용도 등 사용자의 여건에..."
4,최나니,다 좋은데 권한요청 관련 앱 개선 요청 드립니다. 회사 보안 때문에 회사에 들어오면...,5,5/13/2024 7:21,NaN,NaN,다 좋습니다만 권한요청 관련 애플리케이션 개선 요청 드립니다. 회사 보안 때문에 회...


## 한국어 외 텍스트 제거
- 숫자, 특수문자, 영문 등 서비스 경험과 관련된 의미를 추출해 내기 어려운 모든 문자열을 제거합니다.

In [ ]:
review_removed = list(map(lambda review: re.sub('[^가-힣 ]', '', review), dataset['review_prep']))
dataset['review_prep'] = review_removed

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 평점 기준 데이터 분리

In [ ]:
# 긍정적 리뷰(평점 5점 만점 기준 4, 5점)
review_pos = dataset[(4 == dataset['평점']) | (dataset['평점'] == 5)]['review_prep']
# 부정적 리뷰(평점 5점 만점 기준 1, 2점)
review_neg = dataset[(1 == dataset['평점']) | (dataset['평점'] == 2)]['review_prep']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 토큰화
- KoNLPy에서 속도 및 토큰화 측면에서 성능이 가장 우수한 Mecab 형태소 분석기 활용
- 명사가 문장 내 맥락을 파악하는 데 핵심 형태소이며 빈출 어휘를 쉽게 파악하기 위해 명사만 추출

In [ ]:
review_tokenized_pos = list(map(lambda review: mecab.nouns(review), review_pos))
review_tokenized_neg = list(map(lambda review: mecab.nouns(review), review_neg))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 불용어 제거

In [ ]:
def remove_stopword(tokens):
    review_removed_stopword = []
    for token in tokens:
        # 토큰의 글자 수가 2글자 이상인 경우
        if 1 < len(token):
            # 토큰이 불용어가 아닌 경우만 분석용 리뷰 데이터로 포함
            if token not in list(stopword_list['stopword']):
                review_removed_stopword.append(token)
        # 토큰의 글자 수가 1글자인 경우
        else:
            # 1글자 키워드에 포함되는 경우만 분석용 리뷰 데이터로 포함
            if token in list(one_char_keyword['one_char_keyword']):
                review_removed_stopword.append(token)
    return review_removed_stopword

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
review_removed_stopword_pos = list(map(lambda tokens : remove_stopword(tokens), review_tokenized_pos))
review_removed_stopword_neg = list(map(lambda tokens : remove_stopword(tokens), review_tokenized_neg))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 토큰 2 이상 20개 이하인 리뷰만 선별
일반적으로, 리뷰의 길이가 길수록 사용자 경험이나 기술적 문제 등 사용자 의견이 많이 내포되어 있을 가능성이 높습니다. 하지만, 오히려 지나치게 길이가 긴 리뷰는 주제 파악이나 리뷰 내 단어 간의 조합을 활용하여 특징을 추출하는 데 어려움이 있을 수 있습니다(Vasa et al., 2012). 따라서 본 프로젝트에서는 각 리뷰에서 추출된 명사의 개수가 2개 이상 20개 이하인 리뷰만을 분석에 활용하였습니다.

Vasa, R., Hoon, L., Mouzakis, K., & Noguchi, A. (2012, November). A preliminary analysis of mobile app user reviews. In Proceedings of the 24th Australian computer-human interaction conference (pp. 241-244).

In [ ]:
MIN_TOKEN_NUMBER = 2 # 최소 토큰 개수
MAX_TOKEN_NUMBER = 20 # 최대 토큰 개수

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def select_review(review_removed_stopword):
    review_prep = []
    for tokens in review_removed_stopword:
        if MIN_TOKEN_NUMBER <= len(tokens) <= MAX_TOKEN_NUMBER:
            review_prep.append(tokens)
    return review_prep

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
review_prep_pos = select_review(review_removed_stopword_pos)
review_prep_neg = select_review(review_removed_stopword_neg)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 평점별 리뷰 개수

In [ ]:
review_num_pos = len(review_prep_pos)
review_num_neg = len(review_prep_neg)
review_num_tot = review_num_pos + review_num_neg

print(f"분석한 리뷰 총 개수: {review_num_tot}")
print(f"긍정적 리뷰: {review_num_pos}개({(review_num_pos/review_num_tot)*100:.2f}%)")
print(f"부정적 리뷰: {review_num_neg}개({(review_num_neg/review_num_tot)*100:.2f}%)")

분석한 리뷰 총 개수: 279
긍정적 리뷰: 150개(53.76%)
부정적 리뷰: 129개(46.24%)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# LDA 토픽 모델링

## 하이퍼파라미터 튜닝

In [ ]:
NUM_TOPICS = 5 # 토픽 개수는 하이퍼파라미터
# passes: 딥러닝에서 Epoch와 같은 개념으로, 전체 corpus로 모델 학습 횟수 결정
PASSES = 20

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 모델 학습

In [ ]:
def lda_modeling(review_prep):
    # 단어 인코딩 및 빈도수 계산
    dictionary = corpora.Dictionary(review_prep)
    corpus = [dictionary.doc2bow(review) for review in review_prep]
    # LDA 모델 학습
    model = gensim.models.ldamodel.LdaModel(corpus,
                                            num_topics = NUM_TOPICS,
                                            id2word = dictionary,
                                            passes = PASSES)
    return model, corpus, dictionary

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 토픽별 구성 단어 비율 출력

In [ ]:
def print_topic_prop(topics, RATING):
    topic_values = []
    for topic in topics:
        topic_value = topic[1]
        topic_values.append(topic_value)
    topic_prop = pd.DataFrame({"topic_num" : list(range(1, NUM_TOPICS + 1)), "word_prop": topic_values})
    topic_prop.to_excel('/content/result/topic_prop_' + RATING +  '.xlsx')
    display(topic_prop)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## LDA 토픽 모델링 결과 시각화

In [ ]:
def lda_visualize(model, corpus, dictionary, RATING):
    pyLDAvis.enable_notebook()
    result_visualized = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
    pyLDAvis.display(result_visualized)
    # 시각화 결과 저장
    RESULT_FILE = '/content/result/lda_result_' + RATING + '.html'
    pyLDAvis.save_html(result_visualized, RESULT_FILE)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 긍정적 리뷰 토픽 모델링

In [ ]:
model_pos, corpus_pos, dictionary_pos = lda_modeling(review_prep_pos)
NUM_WORDS = 10

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
RATING = 'pos'
topics_pos = model_pos.print_topics(num_words = NUM_WORDS)
print_topic_prop(topics_pos, RATING)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,topic_num,word_prop
0,1,"0.058*""칼로리"" + 0.044*""측정"" + 0.036*""사진"" + 0.036*..."
1,2,"0.063*""운동"" + 0.040*""식단"" + 0.035*""걸음"" + 0.033*""..."
2,3,"0.088*""운동"" + 0.040*""유용"" + 0.028*""모션"" + 0.021*""..."
3,4,"0.153*""건강"" + 0.076*""관리"" + 0.035*""도움"" + 0.032*""..."
4,5,"0.033*""헬스"" + 0.033*""당첨"" + 0.033*""운동"" + 0.029*""..."


In [ ]:
lda_visualize(model_pos, corpus_pos, dictionary_pos, RATING)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 부정적 리뷰 토픽 모델링

In [ ]:
model_neg, corpus_neg, dictionary_neg = lda_modeling(review_prep_neg)
NUM_WORDS = 10

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
RATING = 'neg'
topics_neg = model_neg.print_topics(num_words = NUM_WORDS)
print_topic_prop(topics_neg, RATING)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,topic_num,word_prop
0,1,"0.033*""업데이트"" + 0.030*""챌린지"" + 0.023*""수면"" + 0.02..."
1,2,"0.057*""연동"" + 0.044*""실행"" + 0.042*""설치"" + 0.037*""..."
2,3,"0.034*""미만"" + 0.030*""권한"" + 0.018*""룰렛"" + 0.018*""..."
3,4,"0.057*""로그인"" + 0.028*""화면"" + 0.023*""운동"" + 0.020*..."
4,5,"0.113*""회원"" + 0.076*""가입"" + 0.054*""번호"" + 0.034*""..."


In [ ]:
lda_visualize(model_neg, corpus_neg, dictionary_neg, RATING)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 토픽 모델링 평가: Coherence Scores, Log-likelihood and Perplexity

Log-likelihood, Perplexity and Coherence Score are measures of performance for a topic model. They are used for comparing and discriminating between topic models estimated on the same data. Log-likelihood, perplexity and coherence scores do not have a baseline or a threshold values and therefore are useful only for comparing models.

How do you specify different models? You can set different number of topics and also play with the parameters of the Dirichlet distributions.



#### **Coherence Score**
We will use a function CoherenceModel() from the gensim module (you can also explore that package as it can be used to estimate an LDA model). The sklearn module does not have the functionality to compute the coherence score. Let's install the gensim package and the functions needed:

In [ ]:
!{sys.executable} -m pip install gensim
import gensim

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

/bin/bash: line 1: {sys.executable}: command not found


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Function to normalize text data
def normalize_corpus(corpus):
    # Add your normalization steps (lowercase, remove punctuation, stopwords, stemming/lemmatization, etc.)
    # This is a placeholder for actual normalization logic
    return [gensim.utils.simple_preprocess(doc) for doc in corpus]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Calculate and display coherence score
def calculate_coherence(lda_model, corpus, dictionary, texts):
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print(f'Coherence Score: {coherence_lda}')

calculate_coherence(lda_model=model_pos, corpus=corpus_pos, dictionary=dictionary_pos, texts=review_prep_pos)
calculate_coherence(lda_model=model_neg, corpus=corpus_neg, dictionary=dictionary_neg, texts=review_prep_neg)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Coherence Score: 0.4727767948881305
Coherence Score: 0.45438912914661944


#### **Log-Perplexity Score**

To compute the log-likelihood score we use the **.score** attribute of our defined and fitted LDA function:

In [ ]:
# Calculate and print perplexity instead of log-likelihood
print("Perplexity (lower values are better): ", model_pos.log_perplexity(corpus_pos))
print("Perplexity (lower values are better): ", model_neg.log_perplexity(corpus_neg))

Perplexity (lower values are better):  -5.719880947283654
Perplexity (lower values are better):  -5.792249357485314


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


N-Gram

In [ ]:
# Extract '운동'-related pairs from the bi-gram table of positive sentiments, and then add up the number of counts by each column.
# We can notice that which word is relevant to 'battery'.
from sklearn.feature_extraction.text import CountVectorizer # Import CountVectorizer
vectorizer_Bi_Uni_Grams = CountVectorizer(max_features=1000, ngram_range=(2,2))
Bi_Uni_Grams_matrix = vectorizer_Bi_Uni_Grams.fit_transform(review_pos).toarray()
df = pd.DataFrame(np.round(Bi_Uni_Grams_matrix,2),columns=vectorizer_Bi_Uni_Grams.get_feature_names_out())

# Filter columns containing 'battery'
exercise_columns = [col for col in df.columns if '운동' in col]
exercise_related_data = df[exercise_columns]

# Sum up the counts by each column, and then sort the results out in a descending order.
sums_by_column = exercise_related_data.sum()
sorted_sums = sums_by_column.sort_values(ascending=False)

print(sorted_sums)

건강 아침운동          1
음식 운동기록에         1
운동량이 많으면         1
운동리스트 직접         1
운동시간 입력해서        1
                ..
운동도 하고           1
운동도되고 선물도받고      1
운동도하고 명상도하고      1
운동들을 플레이리스트처럼    1
하루 운동량을          1
Length: 63, dtype: int64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Extract '운동'-related pairs from the bi-gram table of negative sentiments, and then add up the number of counts by each column.
# We can notice that which word is relevant to 'battery'.
vectorizer_Bi_Uni_Grams = CountVectorizer(max_features=1000, ngram_range=(2,2))
Bi_Uni_Grams_matrix = vectorizer_Bi_Uni_Grams.fit_transform(review_neg).toarray()
df = pd.DataFrame(np.round(Bi_Uni_Grams_matrix,2),columns=vectorizer_Bi_Uni_Grams.get_feature_names_out())

# Filter columns containing 'battery'
exercise_columns = [col for col in df.columns if '운동' in col]
exercise_related_data = df[exercise_columns]

# Sum up the counts by each column, and then sort the results out in a descending order.
sums_by_column = exercise_related_data.sum()
sorted_sums = sums_by_column.sort_values(ascending=False)

print(sorted_sums)

운동 시간          1
운동 시작을         1
운동 의욕이         1
운동을 멈추게되요      1
운동을 완료해도       1
운동을 하지말라는건가    1
운동하다 도중에       1
이래선 운동         1
챌린지에 운동        1
초등학생은 운동을      1
dtype: int64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Extract '걸음'-related pairs from the bi-gram table of positive sentiments, and then add up the number of counts by each column.
# We can notice that which word is relevant to 'battery'.
from sklearn.feature_extraction.text import CountVectorizer # Import CountVectorizer
vectorizer_Bi_Uni_Grams = CountVectorizer(max_features=1000, ngram_range=(2,2))
Bi_Uni_Grams_matrix = vectorizer_Bi_Uni_Grams.fit_transform(review_pos).toarray()
df = pd.DataFrame(np.round(Bi_Uni_Grams_matrix,2),columns=vectorizer_Bi_Uni_Grams.get_feature_names_out())

# Filter columns containing 'battery'
exercise_columns = [col for col in df.columns if '걸음' in col]
exercise_related_data = df[exercise_columns]

# Sum up the counts by each column, and then sort the results out in a descending order.
sums_by_column = exercise_related_data.sum()
sorted_sums = sums_by_column.sort_values(ascending=False)

print(sorted_sums)

다르게 걸음없이         1
요즘 만보기걸음에        1
이후 걸음수           1
있는데 걸음수가         1
있습니다신뢰성저하 걸음수    1
있을때만 걸음수가        1
저절로 걸음운동하게       1
종료후 걸음순위확인       1
천걸음 챌린지          1
칼로리걸음운동 항목별      1
특히 걸음            1
한달 천걸음           1
회사에서 걸음챌린지로      1
dtype: int64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Extract '걸음'-related pairs from the bi-gram table of positive sentiments, and then add up the number of counts by each column.
# We can notice that which word is relevant to 'battery'.
from sklearn.feature_extraction.text import CountVectorizer # Import CountVectorizer
vectorizer_Bi_Uni_Grams = CountVectorizer(max_features=1000, ngram_range=(2,2))
Bi_Uni_Grams_matrix = vectorizer_Bi_Uni_Grams.fit_transform(review_neg).toarray()
df = pd.DataFrame(np.round(Bi_Uni_Grams_matrix,2),columns=vectorizer_Bi_Uni_Grams.get_feature_names_out())

# Filter columns containing 'battery'
exercise_columns = [col for col in df.columns if '걸음' in col]
exercise_related_data = df[exercise_columns]

# Sum up the counts by each column, and then sort the results out in a descending order.
sums_by_column = exercise_related_data.sum()
sorted_sums = sums_by_column.sort_values(ascending=False)

print(sorted_sums)

애플리케이션을 걸음챌린지에    1
월일 걸음             1
이번에 걸음를           1
임직원만 걸음를          1
dtype: int64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
